In [ ]:
from src.points import display_week_stats

In [ ]:
leagues = [27465869]
display_week_stats(leagues, league_type='hockey', n_weeks=13, sleep_timeout=7)

In [ ]:
leagues = [8290, 31769, 33730, 52338, 57256, 73362809]
display_week_stats(leagues, league_type='hockey', n_weeks=13, sleep_timeout=7)